In [1]:
# ==============================================================================
# CÉLULA 1: INSTALAÇÃO DE DEPENDÊNCIAS E AUTENTICAÇÃO
# ==============================================================================

print("Instalando dependências...")
!pip install earthengine-api --quiet
!pip install geopandas --quiet
!pip install rasterio --quiet
!pip install shap --quiet
!pip install geemap --quiet
print("Dependências instaladas.")

# ------------------------------------------------------------------------------
# Autenticação e Inicialização de Serviços Google
# ------------------------------------------------------------------------------
from google.colab import drive
import ee

# A autenticação com o Earth Engine pedirá um token de autorização.
try:
    ee.Initialize(project="the-byway-476116-n7")
    print("API do Google Earth Engine já inicializada.")
except Exception as e:
    print("Autenticando na API do Google Earth Engine...")
    ee.Authenticate()
    ee.Initialize(project="the-byway-476116-n7")
    print("API do Google Earth Engine inicializada.")

# Monta o Google Drive para que possamos salvar arquivos (datasets, modelos)
print("Montando Google Drive...")
drive.mount('/content/drive')
print("Google Drive montado em /content/drive")

Instalando dependências...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.5 MB/s eta 0:00:00
Dependências instaladas.
Autenticando na API do Google Earth Engine...
API do Google Earth Engine inicializada.
Montando Google Drive...
Mounted at /content/drive
Google Drive montado em /content/drive


In [2]:
# ==============================================================================
# CÉLULA 2: IMPORTAÇÃO DAS BIBLIOTECAS PRINCIPAIS
# ==============================================================================
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import torch
import geemap

print("Bibliotecas principais importadas. Ambiente pronto para uso!")

Bibliotecas principais importadas. Ambiente pronto para uso!


In [3]:
# ==============================================================================
# PASSO 1: DEFINIÇÃO DOS PARÂMETROS GLOBAIS
# ==============================================================================

# 1. Período de Tempo para Análise(10 anos)
START_DATE = '2014-01-01'
END_DATE = '2023-12-31'

# 2. Área de Interesse(AOI)
# Coleção de biomas do Brasil carregada e filtrada para obter apenas o polígono do Cerrado.
# Este polígono 'cerrado_aoi' será usado para "recortar" todos os outros dados.
biomas = ee.FeatureCollection('projects/mapbiomas-workspace/AUXILIAR/ESTATISTICAS/COLECAO8/VERSAO-1/refined_biome')
cerrado_aoi = biomas.filter(ee.Filter.eq('NAME_PT_BR', 'Cerrado')).geometry()

# 3. Resolução Espacial(30m)
TARGET_RESOLUTION = 30

print("Parâmetros definidos:")
print(f"Período de análise: {START_DATE} a {END_DATE}")
print("Área de Interesse: Bioma Cerrado")

Parâmetros definidos:
Período de análise: 2014-01-01 a 2023-12-31
Área de Interesse: Bioma Cerrado


In [4]:
# ==============================================================================
# PASSO 2: CARREGAR ASSETS DE FOCOS DE CALOR
# ==============================================================================
print("Iniciando o carregamento dos assets de focos de calor...")

# 1. Definição caminho-base (pasta) onde os assets estão.
BASE_PATH = 'projects/the-byway-476116-n7/assets/'

# 2. Crie uma lista com os nomes de todos os seus assets anuais.
asset_names = [
    'bdqueimadas_2014-01-01_2014-12-31',
    'bdqueimadas_2015-01-01_2015-12-31',
    'bdqueimadas_2016-01-01_2016-12-31',
    'bdqueimadas_2017-01-01_2017-12-31',
    'bdqueimadas_2018-01-01_2018-12-31',
    'bdqueimadas_2019-01-01_2019-12-31',
    'bdqueimadas_2020-01-01_2020-12-31',
    'bdqueimadas_2021-01-01_2021-12-31',
    'bdqueimadas_2022-01-01_2022-12-31',
    'bdqueimadas_2023-01-01_2023-12-31'
]

# 3. Crie uma lista VAZIA para armazenar os objetos GEE
list_of_collections = []

# 4. Iteração pela lista de nomes para carregar cada asset
print(f"Encontrados {len(asset_names)} nomes de assets. Carregando do GEE...")

for name in asset_names:
    # Constrói o caminho completo do asset
    full_asset_path = BASE_PATH + name

    # Carrega o asset do GEE como uma FeatureCollection
    fc = ee.FeatureCollection(full_asset_path)

    # Adiciona a coleção carregada à nossa lista
    list_of_collections.append(fc)

print(f"Carregou {len(list_of_collections)} coleções de features com sucesso.")

# 5. Junção de todas as coleções em uma única e grande FeatureCollection
active_fires_collection = ee.FeatureCollection(list_of_collections).flatten()
print(active_fires_collection.first().getInfo())

print("\nAssets anuais de focos de calor mesclados com sucesso no servidor!")
print("O objeto 'active_fires_collection' está pronto.")

Iniciando o carregamento dos assets de focos de calor...
Encontrados 10 nomes de assets. Carregando do GEE...
Carregou 10 coleções de features com sucesso.
{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-44.80399949627662, -11.890001217668136]}, 'id': '0_000000000000000001b7', 'properties': {'Bioma': 'Cerrado', 'DataHora': '2014/01/18 16:39:00', 'DiaSemChuv': 3, 'Estado': 'BAHIA', 'FRP': 8.8, 'Latitude': -11.89, 'Longitude': -44.804, 'Municipio': 'ANGICAL', 'Pais': 'Brasil', 'Precipitac': 1.6, 'RiscoFogo': 0.6, 'Satelite': 'AQUA_M-T'}}

Assets anuais de focos de calor mesclados com sucesso no servidor!
O objeto 'active_fires_collection' está pronto.


In [5]:
# ==============================================================================
# PASSO 3: CARREGAMENTO DAS COLEÇÕES DE FEATURES
# ==============================================================================

# --- 3.1 DADOS METEOROLÓGICOS (ERA5-Land) ---
# Contém dados de temperatura, umidade, vento, etc.
era5_collection = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY') \
   .filterDate(START_DATE, END_DATE) \

print(f"Encontradas {era5_collection.size().getInfo()} imagens horárias de meteorologia.")

# --- 3.2 DADOS DE VEGETAÇÃO (MODIS & Landsat) ---
# NDVI/EVI (saúde da vegetação) e LST (temperatura da superfície) do MODIS.
ndvi_collection = ee.ImageCollection('MODIS/061/MOD13A2') \
   .filterDate(START_DATE, END_DATE) \
   .select('NDVI')

lst_collection = ee.ImageCollection('MODIS/061/MOD11A1') \
   .filterDate(START_DATE, END_DATE) \
   .select('LST_Day_1km')

print(f"Encontradas {ndvi_collection.size().getInfo()} imagens de NDVI (16 dias).")
print(f"Encontradas {lst_collection.size().getInfo()} imagens de LST (diárias).")

# --- 3.3 DADOS TOPOGRÁFICOS (SRTM) ---
# Elevação é uma imagem única (estática), não uma coleção.
elevation = ee.Image('USGS/SRTMGL1_003').select('elevation')
# Cálculo do declive (slope) diretamente a partir da elevação.
slope = ee.Terrain.slope(elevation)

print("Dados de elevação e declive carregados.")

# --- .4 DADOS ANTROPOGÊNICOS (MapBiomas) ---
# Uso e cobertura do solo.
land_cover = ee.Image('projects/mapbiomas-public/assets/brazil/lulc/collection10/mapbiomas_brazil_collection10_coverage_v2')
print(land_cover.bandNames().getInfo())

print("Dados de uso e cobertura do solo carregados.")

Encontradas 87624 imagens horárias de meteorologia.
Encontradas 230 imagens de NDVI (16 dias).
Encontradas 3630 imagens de LST (diárias).
Dados de elevação e declive carregados.
['classification_1985', 'classification_1986', 'classification_1987', 'classification_1988', 'classification_1989', 'classification_1990', 'classification_1991', 'classification_1992', 'classification_1993', 'classification_1994', 'classification_1995', 'classification_1996', 'classification_1997', 'classification_1998', 'classification_1999', 'classification_2000', 'classification_2001', 'classification_2002', 'classification_2003', 'classification_2004', 'classification_2005', 'classification_2006', 'classification_2007', 'classification_2008', 'classification_2009', 'classification_2010', 'classification_2011', 'classification_2012', 'classification_2013', 'classification_2014', 'classification_2015', 'classification_2016', 'classification_2017', 'classification_2018', 'classification_2019', 'classification_

In [6]:
# ==============================================================================
# PASSO 4: ALINHAMENTO ESPAÇO-TEMPORAL (ENRIQUECIMENTO DE DADOS)
# ==============================================================================

# --- 4.1 Amostragem de Preditores Estáticos (Topografia) ---
# (Alinhamento puramente espacial, pois os dados não mudam com o tempo)

print("Iniciando Passo 4.1: Alinhamento Estático (Topografia)...")

# 1. Renomear as bandas (do Passo 3.3) para nomes de colunas claros
#    (A 'elevation' já está selecionada, 'slope' é calculado)
elevation_renamed = elevation.rename('elevation_static')
slope_renamed = slope.rename('slope_static')

# 2. Juntar as duas imagens em uma única imagem "estática" com 2 bandas
static_predictors = ee.Image.cat([
    elevation_renamed,
    slope_renamed
])

# 3. Amostrar os valores dessas bandas para CADA PONTO do 'active_fires_collection'
#    Esta função vai ADICIONAR as novas colunas ('elevation_static' e 'slope_static')
#    a cada ponto de fogo.
dataset_with_static = static_predictors.sampleRegions(
    collection=active_fires_collection,  # Nossa FeatureCollection do Passo 2
    properties=active_fires_collection.first().propertyNames(), # Manter TODAS as colunas do INPE
    scale=TARGET_RESOLUTION,             # Resolução de 30m (do Passo 1)
    geometries=True                      # Manter a geometria (o ponto)
)

print("Alinhamento estático concluído.")
print("A variável 'dataset_with_static' agora contém:")
print("Pontos de Fogo (INPE) + Colunas de 'elevation_static' e 'slope_static'")

Iniciando Passo 4.1: Alinhamento Estático (Topografia)...
Alinhamento estático concluído.
A variável 'dataset_with_static' agora contém:
Pontos de Fogo (INPE) + Colunas de 'elevation_static' e 'slope_static'


In [7]:
# ==============================================================================
# TESTE DA LÓGICA 4.1 (O "TESTE DE VERDADE" - RÁPIDO)
# ==============================================================================
import pprint
print("Iniciando Teste da Lógica 4.1 (com 10 pontos de 2014)...")

try:
    # 1. Crie uma amostra de teste REALMENTE LEVE
    #    Carrega SÓ o asset de 2014 e pega 10 pontos.
    #    Isso é instantâneo.
    asset_2014_path = BASE_PATH + asset_names[0] # Pega o primeiro nome da lista ('...2014...')
    test_collection = ee.FeatureCollection(asset_2014_path).limit(10)

    print("  Amostra de 10 pontos de 2014 carregada.")

    # 2. Recrie as imagens estáticas (globais, sem clip - RÁPIDO)
    elevation = ee.Image('USGS/SRTMGL1_003').select('elevation')
    slope = ee.Terrain.slope(elevation)

    elevation_renamed = elevation.rename('elevation_static')
    slope_renamed = slope.rename('slope_static')
    static_predictors = ee.Image.cat([elevation_renamed, slope_renamed])

    print("  Imagens estáticas (globais) prontas.")

    # 3. Aplique a MESMA LÓGICA do Passo 4.1, mas na coleção de teste LEVE
    test_result = static_predictors.sampleRegions(
        collection=test_collection,  # <-- Usando a coleção LEVE (10 pontos de 2014)
        properties=test_collection.first().propertyNames(), # Pegando as propriedades da coleção LEVE
        scale=TARGET_RESOLUTION,
        geometries=True
    )

    # 4. Chame o .getInfo() no RESULTADO do teste
    print("\nTeste Concluído. Inspecionando o primeiro ponto da amostra:")
    pprint.pprint(test_result.first().getInfo())

    print("\n>>> SUCESSO! As novas colunas 'elevation_static' e 'slope_static' devem estar visíveis.")

except Exception as e:
    print(f"\n>>> FALHA NO TESTE: {e}")

Iniciando Teste da Lógica 4.1 (com 10 pontos de 2014)...
  Amostra de 10 pontos de 2014 carregada.
  Imagens estáticas (globais) prontas.

Teste Concluído. Inspecionando o primeiro ponto da amostra:
{'geometry': {'coordinates': [-44.804003039250375, -11.89010192133555],
              'geodesic': False,
              'type': 'Point'},
 'id': '000000000000000001b7_0',
 'properties': {'Bioma': 'Cerrado',
                'DataHora': '2014/01/18 16:39:00',
                'DiaSemChuv': 3,
                'Estado': 'BAHIA',
                'FRP': 8.8,
                'Latitude': -11.89,
                'Longitude': -44.804,
                'Municipio': 'ANGICAL',
                'Pais': 'Brasil',
                'Precipitac': 1.6,
                'RiscoFogo': 0.6,
                'Satelite': 'AQUA_M-T',
                'elevation_static': 436,
                'slope_static': 1.3258910179138184},
 'type': 'Feature'}

>>> SUCESSO! As novas colunas 'elevation_static' e 'slope_static' devem esta

In [49]:
# ==============================================================================
# PASSO 4.2: ALINHAMENTO ESPAÇO-TEMPORAL (Anual - LULC)
# ==============================================================================
print("Iniciando Passo 4.2: Alinhamento Anual (Cobertura do Solo)...")

# A 'dataset_with_static' é a nossa "receita" do Passo 4.1
# A 'land_cover' é a imagem com bandas anuais (do Passo 3.4)

# 1. Definimos uma função que será aplicada a CADA ponto de fogo
def sample_annual_lulc(feature):
    # 'feature' é o nosso ponto de fogo (que já tem elev/slope)

    # 2. Pega a data do fogo (ex: '2014/01/18 16:39:00')
    date_string = ee.String(feature.get('DataHora'))

    # 3. Extrai o ano (os primeiros 4 caracteres)
    year_string = date_string.slice(0, 4) # Resultado: '2014'

    # 4. Cria o nome da banda do LULC correspondente
    #    ex: 'classification_' + '2014' = 'classification_2014'
    lulc_band_name = ee.String('classification_').cat(year_string)

    # 5. Seleciona APENAS essa banda da imagem de LULC
    lulc_image_this_year = land_cover.select(lulc_band_name)

    # 6. Amostra o valor dessa banda no ponto do 'feature'
    #    O .first() pega o resultado da amostragem (que é uma FeatureCollection)
    lulc_value = lulc_image_this_year.sample(
        region=feature.geometry(), # O Ponto
        scale=TARGET_RESOLUTION,   # 30m
        numPixels=1                # Pega só 1 pixel
    ).first() # Pega o primeiro (e único) resultado

    # 7. Adiciona o valor como uma nova propriedade
    #    Usamos 'ee.Algorithms.If' para evitar erros se o pixel for nulo
    return feature.set('lulc_class', ee.Algorithms.If(
        lulc_value,                                # Se o valor existir...
        lulc_value.get(lulc_band_name),            # ...pegue o valor
        -9999                                      # ...senão, use -9999 (NoData)
    ))

# 8. Aplica a função a TODOS os pontos da nossa coleção
#    Esta é a nova "receita" principal, agora com dados estáticos + LULC
dataset_with_lulc = dataset_with_static.map(sample_annual_lulc)

print("Alinhamento anual (LULC) definido.")
print("A variável 'dataset_with_lulc' agora contém:")
print("Pontos + Topografia + lulc_class")

# (Não vamos testar aqui, pois a 'dataset_with_static' é muito pesada)

Iniciando Passo 4.2: Alinhamento Anual (Cobertura do Solo)...
Alinhamento anual (LULC) definido.
A variável 'dataset_with_lulc' agora contém:
Pontos + Topografia + lulc_class


In [50]:
# ==============================================================================
# PASSO 4.3: ALINHAMENTO ESPAÇO-TEMPORAL (Diário - LST)
# ==============================================================================
print("Iniciando Passo 4.3: Alinhamento Diário (LST)")

# Esta função agora checa por 'null' na DataHora E no pixel amostrado.

def sample_daily_lst(feature):

    # Pega o objeto de data/hora
    datahora_object = feature.get('DataHora')

    # --- Define uma função aninhada que só roda se 'DataHora' NÃO for nulo ---
    def process_with_date(date_obj):
        # --- 1. Processar a Data (sabemos que date_obj não é nulo) ---
        date_string = ee.String(date_obj)
        date_only_string = date_string.slice(0, 10)
        formatted_date_string = date_only_string.replace('/', '-', 'g')
        fire_date = ee.Date(formatted_date_string)

        # --- 2. Encontrar a Imagem ---
        lst_image_this_day = lst_collection.filterDate(fire_date).first()

        # --- 3. Definir a Lógica de Amostragem Segura ---
        def sample_and_scale(image):
            image = ee.Image(image)
            # 'sampled' será uma Feature (se sucesso) ou null (se pixel mascarado)
            sampled = image.sample(
                region=feature.geometry(),
                scale=1000,
                numPixels=1
            ).first()

            # Define a lógica que só roda se 'sampled' NÃO for nulo
            def get_scaled_value(sampled_feature):
                value = ee.Number(sampled_feature.get('LST_Day_1km'))
                return value.multiply(0.02)

            # Checa se 'sampled' é nulo ANTES de chamar .get()
            return ee.Algorithms.If(
                sampled,                      # Se 'sampled' (o pixel) NÃO for nulo
                get_scaled_value(sampled),    # ...pegue o valor
                -9999                         # ...senão (pixel mascarado), retorne NoData
            )

        # Checa se 'lst_image_this_day' é nulo (nenhuma imagem encontrada)
        return ee.Algorithms.If(
            lst_image_this_day,             # Se a imagem existir...
            sample_and_scale(lst_image_this_day), # ...execute a amostragem segura
            -9999                           # ...senão (sem imagem), retorne NoData
        )

    # --- 4. O Cheque Principal (Nível Superior) ---
    lst_value = ee.Algorithms.If(
        datahora_object,                      # Se 'DataHora' NÃO for nulo...
        process_with_date(datahora_object),   # ...execute o processo
        -9999                                 # ...senão (DataHora nula), retorne NoData
    )

    return feature.set('lst_daily_kelvin', lst_value)


# Aplica a função
dataset_with_lst = dataset_with_lulc.map(sample_daily_lst)

print("Alinhamento diário (LST) definido (versão robusta).")
print("A variável 'dataset_with_lst' agora contém:")
print("Pontos + Topografia + LULC + LST (Kelvin)")

Iniciando Passo 4.3: Alinhamento Diário (LST)
Alinhamento diário (LST) definido (versão robusta).
A variável 'dataset_with_lst' agora contém:
Pontos + Topografia + LULC + LST (Kelvin)


In [51]:
# ==============================================================================
# TESTE DA LÓGICA 4.3 (CÉLULA DE TESTE SEGURA)
# ==============================================================================
import pprint
print("Iniciando Teste da Lógica 4.3 (com 10 pontos de 2014)...")

try:
    # 1. Cria uma amostra de teste LEVE (10 pontos de 2014)
    asset_2014_path = BASE_PATH + asset_names[0]
    test_collection = ee.FeatureCollection(asset_2014_path).limit(10)

    # 2. Apliqua a nova função 'sample_daily_lst_robust'
    test_result_lst = test_collection.map(sample_daily_lst)

    # 3. Chama .getInfo() no resultado-teste
    print("\nTeste Concluído. Inspecionando 10 pontos de amostra:")
    pprint.pprint(test_result_lst.getInfo())

    print("\n>>> SUCESSO! A nova coluna 'lst_daily_kelvin' deve estar visível.")
    print("   (Valores -9999 são normais se a DataHora for nula ou o pixel for mascarado)")

except Exception as e:
    print(f"\n>>> FALHA NO TESTE: {e}")

Iniciando Teste da Lógica 4.3 (com 10 pontos de 2014)...

Teste Concluído. Inspecionando 10 pontos de amostra:
{'columns': {'Bioma': 'String',
             'DataHora': 'String',
             'DiaSemChuv': 'Integer',
             'Estado': 'String',
             'FRP': 'Float',
             'Latitude': 'Float',
             'Longitude': 'Float',
             'Municipio': 'String',
             'Pais': 'String',
             'Precipitac': 'Float',
             'RiscoFogo': 'Float',
             'Satelite': 'String',
             'lst_daily_kelvin': 'Number',
             'system:index': 'String'},
 'features': [{'geometry': {'coordinates': [-44.80399949627662,
                                            -11.890001217668136],
                            'type': 'Point'},
               'id': '000000000000000001b7',
               'properties': {'Bioma': 'Cerrado',
                              'DataHora': '2014/01/18 16:39:00',
                              'DiaSemChuv': 3,
              

In [12]:
# ==============================================================================
# PASSO 4.3.V: TESTE DE VALIDAÇÃO VISUAL (LST)
# ==============================================================================
print("Iniciando Teste de Validação Visual para LST...")

# --- 1. Definições ---
DATE_TO_CHECK = '2014-11-21'
# Pegar o primeiro ponto do asset de 2014 (que sabemos ser dessa data)
asset_2014_path = BASE_PATH + asset_names[0]
test_point_to_check = ee.FeatureCollection(asset_2014_path)

# --- 2. Recarregar a coleção LST (do Passo 3.2) ---
#    (Não precisamos do .filterBounds() aqui, é mais rápido)
lst_collection = ee.ImageCollection('MODIS/061/MOD11A1').select('LST_Day_1km')

# --- 3. Pegar a IMAGEM EXATA daquele dia ---
#    .first() pega a única imagem do dia
lst_image_for_day = lst_collection.filterDate(DATE_TO_CHECK).first()

# --- 4. Parâmetros de Visualização (para LST) ---
#    O LST do MODIS (sem escala) vai de ~14500 a ~16000 (para Kelvin 290-320)
#    Usaremos uma paleta de calor (preto-azul-amarelo-vermelho)
lst_vis_params = {
    'min': 14500,  # Valor de 16 bits (bruto)
    'max': 16000,  # Valor de 16 bits (bruto)
    'palette': ['000000', '0000FF', 'FFFF00', 'FF0000']
}

# --- 5. Criar o Mapa ---
Map = geemap.Map()
Map.addLayer(
    lst_image_for_day,
    lst_vis_params,
    f'Imagem LST (MODIS) do dia {DATE_TO_CHECK}'
)
Map.addLayer(
    test_point_to_check,
    {'color': 'cyan'},
    'Ponto de Fogo (INPE)'
)
Map.centerObject(test_point_to_check, 10) # Zoom nível 10
Map.addLayerControl()

print("Mapa criado. Siga as instruções abaixo para inspecionar:")

# --- 6. Exibir o Mapa ---
Map

Iniciando Teste de Validação Visual para LST...
Mapa criado. Siga as instruções abaixo para inspecionar:


Map(center=[-11.209707799168946, -47.38393474734511], controls=(WidgetControl(options=['position', 'transparen…

In [52]:
# ==============================================================================
# PASSO 4.4: ALINHAMENTO ESPAÇO-TEMPORAL (16 Dias - NDVI)
# ==============================================================================
print("Iniciando Passo 4.4: Alinhamento Quinzenal (NDVI Pré-Fogo)...")

def sample_16day_ndvi(feature):

    # Pega o objeto de data/hora
    datahora_object = feature.get('DataHora')

    # --- Define uma função aninhada que só roda se 'DataHora' NÃO for nulo ---
    def process_with_date(date_obj):
        # --- 1. Processar a Data (sabemos que date_obj não é nulo) ---
        date_string = ee.String(date_obj)
        date_only_string = date_string.slice(0, 10)
        formatted_date_string = date_only_string.replace('/', '-', 'g')
        fire_date = ee.Date(formatted_date_string)

        # --- 2. Encontrar a Imagem ---

        # Define um período de busca (ex: os 30 dias anteriores à data do fogo)
        # Isso garante que encontraremos o último compósito de 16 dias.
        search_start_date = fire_date.advance(-30, 'day')
        search_end_date = fire_date # A data do fogo

        # Filtra a coleção pelo período, ordena pela data (mais recente primeiro)
        # e pega o primeiro (que é o compósito imediatamente anterior ao fogo)
        pre_fire_ndvi_image = ndvi_collection \
                                .filterDate(search_start_date, search_end_date) \
                                .sort('system:time_start', False) \
                                .first()

        # --- 3. Definir a Lógica de Amostragem Segura ---
        def sample_and_scale(image):
            image = ee.Image(image)
            sampled = image.sample(
                region=feature.geometry(),
                scale=1000, # Escala nativa do MODIS NDVI (1km)
                numPixels=1
            ).first()

            def get_scaled_value(sampled_feature):
                # O NDVI do MODIS precisa ser multiplicado por 0.0001
                value = ee.Number(sampled_feature.get('NDVI'))
                return value.multiply(0.0001)

            return ee.Algorithms.If(
                sampled,
                get_scaled_value(sampled),
                -9999 # Pixel mascarado (nuvem, etc.)
            )

        # Checa se a imagem pré-fogo foi encontrada
        return ee.Algorithms.If(
            pre_fire_ndvi_image,
            sample_and_scale(pre_fire_ndvi_image),
            -9999 # Nenhuma imagem encontrada no período
        )

    # --- 4. O Cheque Principal (Nível Superior) ---
    ndvi_value = ee.Algorithms.If(
        datahora_object,
        process_with_date(datahora_object),
        -9999 # DataHora era nula
    )

    return feature.set('ndvi_16day_prefire', ndvi_value)


# Aplica a nova função robusta
dataset_with_ndvi = dataset_with_lst.map(sample_16day_ndvi)

print("Alinhamento quinzenal (NDVI) definido.")
print("A variável 'dataset_with_ndvi' agora contém:")
print("Pontos + Topografia + LULC + LST + NDVI (Pré-fogo)")

Iniciando Passo 4.4: Alinhamento Quinzenal (NDVI Pré-Fogo)...
Alinhamento quinzenal (NDVI) definido.
A variável 'dataset_with_ndvi' agora contém:
Pontos + Topografia + LULC + LST + NDVI (Pré-fogo)


In [53]:
# ==============================================================================
# TESTE DA LÓGICA 4.4 (CÉLULA DE TESTE SEGURA)
# ==============================================================================
import pprint
print("Iniciando Teste da Lógica 4.4 (com 10 pontos de 2014)...")

try:
    # 1. Crie uma amostra de teste LEVE (10 pontos de 2014)
    asset_2014_path = BASE_PATH + asset_names[0]
    test_collection = ee.FeatureCollection(asset_2014_path).limit(10)

    # 2. Aplique a NOVA função 'sample_16day_ndvi'
    test_result_ndvi = test_collection.map(sample_16day_ndvi)

    # 3. Chame o .getInfo() no RESULTADO do teste
    print("\nTeste Concluído. Inspecionando 10 pontos de amostra:")
    pprint.pprint(test_result_ndvi.getInfo())

    print("\n>>> SUCESSO! A nova coluna 'ndvi_16day_prefire' deve estar visível.")
    print("   (Valores 'None' ou -9999 são normais se houver nuvens ou dados ausentes)")

except Exception as e:
    print(f"\n>>> FALHA NO TESTE: {e}")

Iniciando Teste da Lógica 4.4 (com 10 pontos de 2014)...

Teste Concluído. Inspecionando 10 pontos de amostra:
{'columns': {'Bioma': 'String',
             'DataHora': 'String',
             'DiaSemChuv': 'Integer',
             'Estado': 'String',
             'FRP': 'Float',
             'Latitude': 'Float',
             'Longitude': 'Float',
             'Municipio': 'String',
             'Pais': 'String',
             'Precipitac': 'Float',
             'RiscoFogo': 'Float',
             'Satelite': 'String',
             'ndvi_16day_prefire': 'Number',
             'system:index': 'String'},
 'features': [{'geometry': {'coordinates': [-44.80399949627662,
                                            -11.890001217668136],
                            'type': 'Point'},
               'id': '000000000000000001b7',
               'properties': {'Bioma': 'Cerrado',
                              'DataHora': '2014/01/18 16:39:00',
                              'DiaSemChuv': 3,
            

In [78]:
# ==============================================================================
# PASSO 4.5: ALINHAMENTO ESPAÇO-TEMPORAL (Horário - ERA5)
# ==============================================================================
print("Iniciando Passo 4.5: Alinhamento Horário (RH + Velocidade do Vento)...")

# 1. Definimos a função que será aplicada a cada ponto
def sample_hourly_era5(feature):

    # 1.A. Atualiza os valores padrão para incluir a velocidade
    default_values = {
        'temp_2m_C': -9999,
        'relative_humidity_percent': -9999,
        'wind_u_ms': -9999,
        'wind_v_ms': -9999,
        'wind_speed_ms': -9999,
        'precipitation_hourly_mm': -9999
    }

    datahora_object = feature.get('DataHora')

    def process_with_date(date_obj):
        date_string = ee.String(date_obj)
        iso_string_step1 = date_string.replace('/', '-', 'g')
        iso_string_final = iso_string_step1.replace(' ', 'T')
        fire_datetime = ee.Date(iso_string_final)

        # Lógica de join correta (pega a hora mais próxima antes)
        search_start_date = fire_datetime.advance(-2, 'hour')
        search_end_date = fire_datetime
        era5_image_this_hour = era5_collection \
                                .filterDate(search_start_date, search_end_date) \
                                .sort('system:time_start', False) \
                                .first()

        def sample_and_scale(image):
            image = ee.Image(image)
            bands_to_sample = [
                'temperature_2m',
                'dewpoint_temperature_2m',
                'u_component_of_wind_10m',
                'v_component_of_wind_10m',
                'total_precipitation_hourly'
            ]

            sampled = image.select(bands_to_sample).sample(
                region=feature.geometry(), scale=11132, numPixels=1
            ).first()

            # ==================================================
            # CÁLCULO DE RH E VELOCIDADE
            # ==================================================
            def get_scaled_values(sampled_feature):
                # 1. Pega os valores brutos
                temp_k = ee.Number(sampled_feature.get('temperature_2m'))
                dew_k = ee.Number(sampled_feature.get('dewpoint_temperature_2m'))
                wind_u = ee.Number(sampled_feature.get('u_component_of_wind_10m'))
                wind_v = ee.Number(sampled_feature.get('v_component_of_wind_10m'))
                precip_m = ee.Number(sampled_feature.get('total_precipitation_hourly'))

                # 2. Converte para Celsius
                temp_c = temp_k.subtract(273.15)
                dew_c = dew_k.subtract(273.15)

                # 3. Calcula RH (%) usando a Equação de Magnus
                Ea = ee.Number(6.1094).multiply(
                    (ee.Number(17.625).multiply(dew_c)).divide(dew_c.add(243.04)).exp()
                )
                Es = ee.Number(6.1094).multiply(
                    (ee.Number(17.625).multiply(temp_c)).divide(temp_c.add(243.04)).exp()
                )
                rh = Ea.divide(Es).multiply(100)

                # 4. CALCULA A VELOCIDADE DO VENTO (Wind Speed)
                # wind_speed = sqrt(wind_u^2 + wind_v^2)
                wind_speed = wind_u.pow(2).add(wind_v.pow(2)).sqrt()

                # 5. Retorna o dicionário com TODAS as features
                return {
                    'temp_2m_C': temp_c,
                    'relative_humidity_percent': rh,
                    'wind_u_ms': wind_u,   # Componente U (para direção)
                    'wind_v_ms': wind_v,   # Componente V (para direção)
                    'wind_speed_ms': wind_speed, # Velocidade total (magnitude)
                    'precipitation_hourly_mm': precip_m.multiply(1000)
                }

            return ee.Algorithms.If(sampled, get_scaled_values(sampled), default_values)

        return ee.Algorithms.If(era5_image_this_hour, sample_and_scale(era5_image_this_hour), default_values)

    final_properties = ee.Algorithms.If(
        datahora_object,
        process_with_date(datahora_object),
        default_values
    )

    return feature.set(final_properties)


# 5. Aplica a nova função
dataset_final_positivo = dataset_with_ndvi.map(sample_hourly_era5)

print("Alinhamento horário (ERA5) [VERSÃO FINAL COMPLETA] definido.")
print("="*60)
print("FASE 2.2 (ALINHAMENTO) CONCLUÍDA!")
print("A variável 'dataset_final_positivo' contém a 'receita' final.")
print("Pontos + Topo + LULC + LST + NDVI + Meteo (RH, U, V, Velocidade)")
print("="*60)

Iniciando Passo 4.5: Alinhamento Horário (RH + Velocidade do Vento)...
Alinhamento horário (ERA5) [VERSÃO FINAL COMPLETA] definido.
FASE 2.2 (ALINHAMENTO) CONCLUÍDA!
A variável 'dataset_final_positivo' contém a 'receita' final.
Pontos + Topo + LULC + LST + NDVI + Meteo (RH, U, V, Velocidade)


In [79]:
# ==============================================================================
# TESTE DE AMOSTRAGEM: 10 PONTOS DE BRASÍLIA (do asset de 2014)
# ==============================================================================
import pprint
print("Iniciando Teste de Amostragem em Brasília (dados de 2014)...")

try:
    # 1. Define o filtro para 'BRASILIA'
    brasilia_filter = ee.Filter.eq('Municipio', 'BRAS�LIA')

    # 2. Carrega APENAS o asset de 2014 (para evitar timeout)
    asset_2014_path = BASE_PATH + asset_names[0]

    # 3. Filtra o asset de 2014 e pega 10 pontos
    test_collection_brasilia = ee.FeatureCollection(asset_2014_path) \
                                    .filter(brasilia_filter) \
                                    .limit(10)

    # 4. Checa (de forma leve) se encontrou algum ponto
    test_size = test_collection_brasilia.size().getInfo()

    if test_size == 0:
        print("\n>>> AVISO: Nenhum ponto de fogo encontrado para 'BRASILIA' no asset de 2014.")
        print("    Isso não é um erro. Tente mudar o índice do asset (ex: asset_names[5] para 2019)")

    else:
        print(f"Encontrados {test_size} pontos em Brasília (no asset 2014).")
        print("Aplicando lógica de amostragem completa (Passos 4.1 a 4.5)...")

        # 5. Aplica TODAS as lógicas de amostragem (as funções devem estar na memória)

        # Passo 4.1 (Estático) - (static_predictors deve existir)
        test_result_static = static_predictors.sampleRegions(
            collection=test_collection_brasilia,
            properties=test_collection_brasilia.first().propertyNames(),
            scale=TARGET_RESOLUTION,
            geometries=True
        )

        # Passo 4.2 (LULC) - (função sample_annual_lulc)
        test_result_lulc = test_result_static.map(sample_annual_lulc)

        # Passo 4.3 (LST) - (função sample_daily_lst)
        test_result_lst = test_result_lulc.map(sample_daily_lst)

        # Passo 4.4 (NDVI) - (função sample_16day_ndvi)
        test_result_ndvi = test_result_lst.map(sample_16day_ndvi)

        # Passo 4.5 (ERA5) - (função sample_hourly_era5)
        final_test_result_brasilia = test_result_ndvi.map(sample_hourly_era5)

        # 6. Chame o .getInfo() no RESULTADO do teste final
        print("\nTeste Concluído. Inspecionando pontos de amostra de Brasília:")
        pprint.pprint(final_test_result_brasilia.getInfo())
        print("\n>>> SUCESSO! Todos os dados alinhados para Brasília.")

except Exception as e:
    print(f"\n>>> FALHA NO TESTE DE BRASÍLIA: {e}")
    print("   (Verifique se 'BRASILIA' é o nome correto no seu shapefile)")

Iniciando Teste de Amostragem em Brasília (dados de 2014)...
Encontrados 10 pontos em Brasília (no asset 2014).
Aplicando lógica de amostragem completa (Passos 4.1 a 4.5)...

Teste Concluído. Inspecionando pontos de amostra de Brasília:
{'columns': {},
 'features': [{'geometry': {'coordinates': [-47.51700502881974,
                                            -15.622871421437402],
                            'geodesic': False,
                            'type': 'Point'},
               'id': '00000000000000000325_0',
               'properties': {'Bioma': 'Cerrado',
                              'DataHora': '2014/02/19 16:39:00',
                              'DiaSemChuv': 5,
                              'Estado': 'DISTRITO FEDERAL',
                              'FRP': 11.7,
                              'Latitude': -15.623,
                              'Longitude': -47.517,
                              'Municipio': 'BRAS�LIA',
                              'Pais': 'Brasil',
      

In [44]:
# ==============================================================================
# PASSO 4.5: TESTE VISUAL (A "PROVA" DOS VALORES)
# ==============================================================================
import geemap
import pprint
print("Iniciando Teste Visual (Prova dos Valores)...")

# --- 1. Definir os EXATOS valores do nosso último teste ---
test_point = ee.Geometry.Point([-44.804, -11.89])
test_datetime = '2014-01-18T16:00:00'
band_to_check = 'temperature_2m'

print(f"Verificando a banda '{band_to_check}' no ponto [-44.80, -11.89] em {test_datetime}")

# --- 2. Carregar a Imagem Exata ---
era5_collection = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')
image_this_hour = era5_collection \
                    .filterDate(test_datetime) \
                    .first()

# --- 3. Parâmetros de Visualização (para Temperatura em Kelvin) ---
era5_vis_params = {
    'min': 290,  # Aprox 17°C
    'max': 315,  # Aprox 42°C
    'palette': ['0000FF', 'FFFF00', 'FF0000'] # Azul -> Amarelo -> Vermelho
}

# --- 4. Criar o Mapa ---
Map = geemap.Map()
Map.addLayer(
    image_this_hour.select(band_to_check),
    era5_vis_params,
    f'ERA5 ({band_to_check}) em {test_datetime}'
)
Map.addLayer(
    test_point,
    {'color': 'cyan', 'pointSize': 5},
    'Ponto de Fogo (INPE)'
)
Map.centerObject(test_point, 9) # Zoom 9 (para ver os pixels grandes)
Map.addLayerControl()

print("Mapa criado. Siga as instruções abaixo:")

# --- 5. Exibir o Mapa ---
Map

Iniciando Teste Visual (Prova dos Valores)...
Verificando a banda 'temperature_2m' no ponto [-44.80, -11.89] em 2014-01-18T16:00:00
Mapa criado. Siga as instruções abaixo:


Map(center=[-11.890000000000002, -44.803999999999995], controls=(WidgetControl(options=['position', 'transpare…